<a href="https://colab.research.google.com/github/CVL1971/k-diffusion/blob/master/Pro_Deforum_Stable_Diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Pro Deforum Stable Diffusion v0.3**
[Stable Diffusion](https://github.com/CompVis/stable-diffusion) by Robin Rombach, Andreas Blattmann, Dominik Lorenz, Patrick Esser, Björn Ommer and the [Stability.ai](https://stability.ai/) Team. [K Diffusion](https://github.com/crowsonkb/k-diffusion) by [Katherine Crowson](https://twitter.com/RiversHaveWings). You need to get the ckpt file and put it on your Google Drive first to use this. It can be downloaded from [HuggingFace](https://huggingface.co/CompVis/stable-diffusion).

Notebook by [deforum](https://discord.gg/upmXXsrwZc)

# Setup

In [ ]:
#@markdown **NVIDIA GPU**
import subprocess
sub_p_res = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total,memory.free', '--format=csv,noheader'], stdout=subprocess.PIPE).stdout.decode('utf-8')
print(sub_p_res)

In [ ]:
#@markdown **Model and Output Paths**
# ask for the link
print("Local Path Variables:\n")

models_path = "/content/models" #@param {type:"string"}
output_path = "/content/output" #@param {type:"string"}
temporal_path= "/content/temporal" #@param {type:"string"}

#@markdown **Google Drive Path Variables (Optional)**
mount_google_drive = True #@param {type:"boolean"}
force_remount = False

if mount_google_drive:
    from google.colab import drive # type: ignore
    try:
        drive_path = "/content/drive"
        drive.mount(drive_path,force_remount=force_remount)
        models_path_gdrive = "/content/drive/MyDrive/AI/models" #@param {type:"string"}
        output_path_gdrive = "/content/drive/MyDrive/AI/StableDiffusion" #@param {type:"string"}
        models_path = models_path_gdrive
        output_path = output_path_gdrive
    except:
        print("...error mounting drive or with drive path variables")
        print("...reverting to default path variables")

import os
os.makedirs(models_path, exist_ok=True)
os.makedirs(output_path, exist_ok=True)
os.makedirs(temporal_path, exist_ok=True)

print(f"models_path: {models_path}")
print(f"output_path: {output_path}")
print(f"temporal_path: {temporal_path}")

In [ ]:
%%capture
#@markdown **Setup Environment**

setup_environment = True #@param {type:"boolean"}
print_subprocess = False #@param {type:"boolean"}



if setup_environment:
    import subprocess, time
    print("Setting up environment...")
    start_time = time.time()
    all_process = [
        ['pip', 'install', 'pycryptodome'],
        ['pip', 'install', 'torch==1.12.1+cu113', 'torchvision==0.13.1+cu113', '--extra-index-url', 'https://download.pytorch.org/whl/cu113'],
        ['pip', 'install', 'omegaconf==2.2.3', 'einops==0.4.1', 'pytorch-lightning==1.7.4', 'torchmetrics==0.9.3', 'torchtext==0.13.1', 'transformers==4.21.2', 'kornia==0.6.7'],
        #['git', 'clone', 'https://github.com/deforum/stable-diffusion'],
        ['git', 'clone', 'https://github.com/CVL1971/stable-diffusion'],
        ['pip', 'install', '-e', 'git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers'],
        ['pip', 'install', '-e', 'git+https://github.com/openai/CLIP.git@main#egg=clip'],
        ['pip', 'install', 'accelerate', 'ftfy', 'jsonmerge', 'matplotlib', 'resize-right', 'timm', 'torchdiffeq'],
        ['git', 'clone', 'https://github.com/shariqfarooq123/AdaBins.git'],
        ['git', 'clone', 'https://github.com/isl-org/MiDaS.git'],
        ['git', 'clone', 'https://github.com/MSFTserver/pytorch3d-lite.git'],
    ]
    for process in all_process:
        running = subprocess.run(process,stdout=subprocess.PIPE).stdout.decode('utf-8')
        if print_subprocess:
            print(running)
    
    print(subprocess.run(['git', 'clone', 'https://github.com/CVL1971/k-diffusion/'], stdout=subprocess.PIPE).stdout.decode('utf-8'))
    with open('k-diffusion/k_diffusion/__init__.py', 'w') as f:
        f.write('')

    end_time = time.time()
    print(f"Environment set up in {end_time-start_time:.0f} seconds")

In [ ]:
#@markdown **Python Definitions**
import json
from IPython import display
import math, os, pathlib, shutil, subprocess, sys, time
import cv2
import numpy as np
import pandas as pd
import random
import requests
import torch, torchvision
import torch.nn as nn
import torchvision.transforms as T
import torchvision.transforms.functional as TF
from contextlib import contextmanager, nullcontext
from einops import rearrange, repeat
from itertools import islice
from omegaconf import OmegaConf
from PIL import Image
from pytorch_lightning import seed_everything
from skimage.exposure import match_histograms
from torchvision.utils import make_grid
from tqdm import tqdm, trange
from types import SimpleNamespace
from torch import autocast

sys.path.extend([
    'src/taming-transformers',
    'src/clip',
    'stable-diffusion/',
    'k-diffusion',
    'pytorch3d-lite',
    'AdaBins',
    'MiDaS',
])

import py3d_tools as p3d
from helpers import save_samples, sampler_fn
from infer import InferenceHelper
from k_diffusion import sampling
from k_diffusion.external import CompVisDenoiser
from ldm.util import instantiate_from_config
from ldm.models.diffusion.ddim import DDIMSampler
from ldm.models.diffusion.plms import PLMSSampler
from midas.dpt_depth import DPTDepthModel
from midas.transforms import Resize, NormalizeImage, PrepareForNet

#123
#def tensor_to_image(tensor):
#    tensor = tensor*255
#    tensor = np.array(tensor, dtype=np.uint8)
#    if np.ndim(tensor)>3:
#        assert tensor.shape[0] == 1
#        tensor = tensor[0]
#    return PIL.Image.fromarray(tensor)




def sanitize(prompt):
    whitelist = set('abcdefghijklmnopqrstuvwxyz ABCDEFGHIJKLMNOPQRSTUVWXYZ')
    tmp = ''.join(filter(whitelist.__contains__, prompt))
    return '_'.join(prompt.split(" "))

def anim_frame_warp_2d(prev_img_cv2, args, anim_args, keys, frame_idx):
    angle = keys.angle_series[frame_idx]
    zoom = keys.zoom_series[frame_idx]
    translation_x = keys.translation_x_series[frame_idx]
    translation_y = keys.translation_y_series[frame_idx]

    center = (args.W // 2, args.H // 2)
    trans_mat = np.float32([[1, 0, translation_x], [0, 1, translation_y]])
    rot_mat = cv2.getRotationMatrix2D(center, angle, zoom)
    trans_mat = np.vstack([trans_mat, [0,0,1]])
    rot_mat = np.vstack([rot_mat, [0,0,1]])
    xform = np.matmul(rot_mat, trans_mat)

    return cv2.warpPerspective(
        prev_img_cv2,
        xform,
        (prev_img_cv2.shape[1], prev_img_cv2.shape[0]),
        borderMode=cv2.BORDER_WRAP if anim_args.border == 'wrap' else cv2.BORDER_REPLICATE
    )

def anim_frame_warp_3d(prev_img_cv2, anim_args, keys, frame_idx, adabins_helper, midas_model, midas_transform):
    TRANSLATION_SCALE = 1.0/200.0 # matches Disco
    translate_xyz = [
        -keys.translation_x_series[frame_idx] * TRANSLATION_SCALE, 
        keys.translation_y_series[frame_idx] * TRANSLATION_SCALE, 
        -keys.translation_z_series[frame_idx] * TRANSLATION_SCALE
    ]
    rotate_xyz = [
        math.radians(keys.rotation_3d_x_series[frame_idx]), 
        math.radians(keys.rotation_3d_y_series[frame_idx]), 
        math.radians(keys.rotation_3d_z_series[frame_idx])
    ]
    rot_mat = p3d.euler_angles_to_matrix(torch.tensor(rotate_xyz, device=device), "XYZ").unsqueeze(0)
    result = transform_image_3d(prev_img_cv2, adabins_helper, midas_model, midas_transform, rot_mat, translate_xyz, anim_args)
    torch.cuda.empty_cache()
    return result

def add_noise(sample: torch.Tensor, noise_amt: float):
    return sample + torch.randn(sample.shape, device=sample.device) * noise_amt

def download_depth_models():
    def wget(url, outputdir):
        print(subprocess.run(['wget', url, '-P', outputdir], stdout=subprocess.PIPE).stdout.decode('utf-8'))
    if not os.path.exists(os.path.join(models_path, 'dpt_large-midas-2f21e586.pt')):
        print("Downloading dpt_large-midas-2f21e586.pt...")
        wget("https://github.com/intel-isl/DPT/releases/download/1_0/dpt_large-midas-2f21e586.pt", models_path)
    if not os.path.exists('pretrained/AdaBins_nyu.pt'):
        print("Downloading AdaBins_nyu.pt...")
        os.makedirs('pretrained', exist_ok=True)
        wget("https://cloudflare-ipfs.com/ipfs/Qmd2mMnDLWePKmgfS8m6ntAg4nhV5VkUyAydYBp8cWWeB7/AdaBins_nyu.pt", 'pretrained')

def get_output_folder(output_path, batch_folder):
    out_path = os.path.join(output_path,time.strftime('%Y-%m'))
    if batch_folder != "":
        out_path = os.path.join(out_path, batch_folder)
    os.makedirs(out_path, exist_ok=True)
    return out_path

def load_depth_model(optimize=True):
    midas_model = DPTDepthModel(
        path=f"{models_path}/dpt_large-midas-2f21e586.pt",
        backbone="vitl16_384",
        non_negative=True,
    )
    normalization = NormalizeImage(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])

    midas_transform = T.Compose([
        Resize(
            384, 384,
            resize_target=None,
            keep_aspect_ratio=True,
            ensure_multiple_of=32,
            resize_method="minimal",
            image_interpolation_method=cv2.INTER_CUBIC,
        ),
        normalization,
        PrepareForNet()
    ])

    midas_model.eval()    
    if optimize:
        if device == torch.device("cuda"):
            midas_model = midas_model.to(memory_format=torch.channels_last)
            midas_model = midas_model.half()
    midas_model.to(device)

    return midas_model, midas_transform

def load_img(path, shape):
    if path.startswith('http://') or path.startswith('https://'):
        image = Image.open(requests.get(path, stream=True).raw).convert('RGB')
    else:
        image = Image.open(path).convert('RGB')

    image = image.resize(shape, resample=Image.LANCZOS)
    image = np.array(image).astype(np.float16) / 255.0
    image = image[None].transpose(0, 3, 1, 2)
    image = torch.from_numpy(image)
    return 2.*image - 1.

def load_mask_img(path, shape):
    # path (str): Path to the mask image
    # shape (list-like len(4)): shape of the image to match, usually latent_image.shape
    mask_w_h = (shape[-1], shape[-2])
    if path.startswith('http://') or path.startswith('https://'):
        mask_image = Image.open(requests.get(path, stream=True).raw).convert('RGBA')
    else:
        mask_image = Image.open(path).convert('RGBA')
    mask = mask_image.resize(mask_w_h, resample=Image.LANCZOS)
    mask = mask.convert("L")
    return mask

def maintain_colors(prev_img, color_match_sample, mode):
    if mode == 'Match Frame 0 RGB':
        return match_histograms(prev_img, color_match_sample, multichannel=True)
    elif mode == 'Match Frame 0 HSV':
        prev_img_hsv = cv2.cvtColor(prev_img, cv2.COLOR_RGB2HSV)
        color_match_hsv = cv2.cvtColor(color_match_sample, cv2.COLOR_RGB2HSV)
        matched_hsv = match_histograms(prev_img_hsv, color_match_hsv, multichannel=True)
        return cv2.cvtColor(matched_hsv, cv2.COLOR_HSV2RGB)
    else: # Match Frame 0 LAB
        prev_img_lab = cv2.cvtColor(prev_img, cv2.COLOR_RGB2LAB)
        color_match_lab = cv2.cvtColor(color_match_sample, cv2.COLOR_RGB2LAB)
        matched_lab = match_histograms(prev_img_lab, color_match_lab, multichannel=True)
        return cv2.cvtColor(matched_lab, cv2.COLOR_LAB2RGB)


def make_callback(sampler_name, dynamic_threshold=None, static_threshold=None, mask=None, init_latent=None, sigmas=None, sampler=None, masked_noise_modifier=1.0, showdiffusion=None, savediffusion=None, listsaveddiffusion=None, skipsamples=None, countsamples=None):  
    display.display('make_callback')
    # Creates the callback function to be passed into the samplers
    # The callback function is applied to the image at each step
    def dynamic_thresholding_(img, threshold):
        #123
        display.display('dynamic_thresholding_')
        # Dynamic thresholding from Imagen paper (May 2022)
        s = np.percentile(np.abs(img.cpu()), threshold, axis=tuple(range(1,img.ndim)))
        s = np.max(np.append(s,1.0))
        torch.clamp_(img, -1*s, s)
        torch.FloatTensor.div_(img, s)
      
      

    # Callback for samplers in the k-diffusion repo, called thus:
    #   callback({'x': x, 'i': i, 'sigma': sigmas[i], 'sigma_hat': sigmas[i], 'denoised': denoised})
    def k_callback_(args_dict):
        #123
        #456
        
        
  
       
        #display.display(vimage)
        # vfilename=time.strftime('%Y-%m-%H-%M-%S') + '.PNG'
        # vfileoutput=os.path.join('/content/drive/MyDrive/AI/StableDiffusion',vfilename)
        # vimage.save(vfileoutput)
        # tfile=open(vfileoutput, "rb")
        # wimgresult.value=tfile.read()

        brest=(countsamples.value() % skipsamples)
        if ((showdiffusion or savediffusion) and (brest==0)):
          v1=model.decode_first_stage(args_dict['denoised'])
          imgin8=sample_to_cv2(v1)
          vimage=Image.fromarray(imgin8)

          if (showdiffusion):
              #print (countsamples.value())
              buf = io.BytesIO()
              vimage.save(buf, format='PNG')
              wimgresult.value=buf.getvalue()
              buf.flush
              buf.close
             
              if savediffusion:
                listsaveddiffusion.append(vimage)
              else:
                vimage.close
            
        countsamples.inc()

        if dynamic_threshold is not None:
            dynamic_thresholding_(args_dict['x'], dynamic_threshold)
            print("dynamic")
        if static_threshold is not None:
            torch.clamp_(args_dict['x'], -1*static_threshold, static_threshold)
            print("static")
        if mask is not None:
            init_noise = init_latent + noise * args_dict['sigma']
            is_masked = torch.logical_and(mask >= mask_schedule[args_dict['i']], mask != 0 )
            new_img = init_noise * torch.where(is_masked,1,0) + args_dict['x'] * torch.where(is_masked,0,1)
            args_dict['x'].copy_(new_img)



    # Function that is called on the image (img) and step (i) at each step
    def img_callback_(img, i):
        #123
        display.display('img_callback_')
        #print(f"1")
        #img2=tensor_to_image(img.cpu())
        #img2.save("hello")
        # Thresholding functions
        
        if dynamic_threshold is not None:
            dynamic_thresholding_(img, dynamic_threshold)
        if static_threshold is not None:
            torch.clamp_(img, -1*static_threshold, static_threshold)
        if mask is not None:
            i_inv = len(sigmas) - i - 1
            init_noise = sampler.stochastic_encode(init_latent, torch.tensor([i_inv]*batch_size).to(device), noise=noise)
            is_masked = torch.logical_and(mask >= mask_schedule[i], mask != 0 )
            new_img = init_noise * torch.where(is_masked,1,0) + img * torch.where(is_masked,0,1)
            img.copy_(new_img)
                   
    if init_latent is not None:
        noise = torch.randn_like(init_latent, device=device) * masked_noise_modifier
    if sigmas is not None and len(sigmas) > 0:
        mask_schedule, _ = torch.sort(sigmas/torch.max(sigmas))
    elif len(sigmas) == 0:
        mask = None # no mask needed if no steps (usually happens because strength==1.0)
    if sampler_name in ["plms","ddim"]: 
        # Callback function formated for compvis latent diffusion samplers
        if mask is not None:
            assert sampler is not None, "Callback function for stable-diffusion samplers requires sampler variable"
            batch_size = init_latent.shape[0]

        callback = img_callback_
    else: 
        # Default callback function uses k-diffusion sampler variables
        callback = k_callback_

    return callback

def prepare_mask(mask_file, mask_shape, mask_brightness_adjust=1.0, mask_contrast_adjust=1.0):
    # path (str): Path to the mask image
    # shape (list-like len(4)): shape of the image to match, usually latent_image.shape
    # mask_brightness_adjust (non-negative float): amount to adjust brightness of the iamge, 
    #     0 is black, 1 is no adjustment, >1 is brighter
    # mask_contrast_adjust (non-negative float): amount to adjust contrast of the image, 
    #     0 is a flat grey image, 1 is no adjustment, >1 is more contrast
                            
    mask = load_mask_img(mask_file, mask_shape)

    # Mask brightness/contrast adjustments
    if mask_brightness_adjust != 1:
        mask = TF.adjust_brightness(mask, mask_brightness_adjust)
    if mask_contrast_adjust != 1:
        mask = TF.adjust_contrast(mask, mask_contrast_adjust)

    # Mask image to array
    mask = np.array(mask).astype(np.float32) / 255.0
    mask = np.tile(mask,(4,1,1))
    mask = np.expand_dims(mask,axis=0)
    mask = torch.from_numpy(mask)

    if args.invert_mask:
        mask = ( (mask - 0.5) * -1) + 0.5
    
    mask = np.clip(mask,0,1)
    return mask

def sample_from_cv2(sample: np.ndarray) -> torch.Tensor:
    sample = ((sample.astype(float) / 255.0) * 2) - 1
    sample = sample[None].transpose(0, 3, 1, 2).astype(np.float16)
    sample = torch.from_numpy(sample)
    return sample

def sample_to_cv2(sample: torch.Tensor) -> np.ndarray:
    sample_f32 = rearrange(sample.squeeze().cpu().numpy(), "c h w -> h w c").astype(np.float32)
    sample_f32 = ((sample_f32 * 0.5) + 0.5).clip(0, 1)
    sample_int8 = (sample_f32 * 255).astype(np.uint8)
    return sample_int8

@torch.no_grad()
def transform_image_3d(prev_img_cv2, adabins_helper, midas_model, midas_transform, rot_mat, translate, anim_args):
    # adapted and optimized version of transform_image_3d from Disco Diffusion https://github.com/alembics/disco-diffusion 

    w, h = prev_img_cv2.shape[1], prev_img_cv2.shape[0]

    # predict depth with AdaBins    
    use_adabins = anim_args.midas_weight < 1.0 and adabins_helper is not None
    if use_adabins:
        print(f"Estimating depth of {w}x{h} image with AdaBins...")
        MAX_ADABINS_AREA = 500000
        MIN_ADABINS_AREA = 448*448

        # resize image if too large or too small
        img_pil = Image.fromarray(cv2.cvtColor(prev_img_cv2, cv2.COLOR_RGB2BGR))
        image_pil_area = w*h
        resized = True
        if image_pil_area > MAX_ADABINS_AREA:
            scale = math.sqrt(MAX_ADABINS_AREA) / math.sqrt(image_pil_area)
            depth_input = img_pil.resize((int(w*scale), int(h*scale)), Image.LANCZOS) # LANCZOS is good for downsampling
            print(f"  resized to {depth_input.width}x{depth_input.height}")
        elif image_pil_area < MIN_ADABINS_AREA:
            scale = math.sqrt(MIN_ADABINS_AREA) / math.sqrt(image_pil_area)
            depth_input = img_pil.resize((int(w*scale), int(h*scale)), Image.BICUBIC)
            print(f"  resized to {depth_input.width}x{depth_input.height}")
        else:
            depth_input = img_pil
            resized = False

        # predict depth and resize back to original dimensions
        try:
            _, adabins_depth = adabins_helper.predict_pil(depth_input)
            if resized:
                adabins_depth = torchvision.transforms.functional.resize(
                    torch.from_numpy(adabins_depth), 
                    torch.Size([h, w]),
                    interpolation=torchvision.transforms.functional.InterpolationMode.BICUBIC
                )
            adabins_depth = adabins_depth.squeeze()
        except:
            print(f"  exception encountered, falling back to pure MiDaS")
            use_adabins = False
        torch.cuda.empty_cache()

    if midas_model is not None:
        # convert image from 0->255 uint8 to 0->1 float for feeding to MiDaS
        img_midas = prev_img_cv2.astype(np.float32) / 255.0
        img_midas_input = midas_transform({"image": img_midas})["image"]

        # MiDaS depth estimation implementation
        print(f"Estimating depth of {w}x{h} image with MiDaS...")
        sample = torch.from_numpy(img_midas_input).float().to(device).unsqueeze(0)
        if device == torch.device("cuda"):
            sample = sample.to(memory_format=torch.channels_last)  
            sample = sample.half()
        midas_depth = midas_model.forward(sample)
        midas_depth = torch.nn.functional.interpolate(
            midas_depth.unsqueeze(1),
            size=img_midas.shape[:2],
            mode="bicubic",
            align_corners=False,
        ).squeeze()
        midas_depth = midas_depth.cpu().numpy()
        torch.cuda.empty_cache()

        # MiDaS makes the near values greater, and the far values lesser. Let's reverse that and try to align with AdaBins a bit better.
        midas_depth = np.subtract(50.0, midas_depth)
        midas_depth = midas_depth / 19.0

        # blend between MiDaS and AdaBins predictions
        if use_adabins:
            depth_map = midas_depth*anim_args.midas_weight + adabins_depth*(1.0-anim_args.midas_weight)
        else:
            depth_map = midas_depth

        depth_map = np.expand_dims(depth_map, axis=0)
        depth_tensor = torch.from_numpy(depth_map).squeeze().to(device)
    else:
        depth_tensor = torch.ones((h, w), device=device)

    pixel_aspect = 1.0 # aspect of an individual pixel (so usually 1.0)
    near, far, fov_deg = anim_args.near_plane, anim_args.far_plane, anim_args.fov
    persp_cam_old = p3d.FoVPerspectiveCameras(near, far, pixel_aspect, fov=fov_deg, degrees=True, device=device)
    persp_cam_new = p3d.FoVPerspectiveCameras(near, far, pixel_aspect, fov=fov_deg, degrees=True, R=rot_mat, T=torch.tensor([translate]), device=device)

    # range of [-1,1] is important to torch grid_sample's padding handling
    y,x = torch.meshgrid(torch.linspace(-1.,1.,h,dtype=torch.float32,device=device),torch.linspace(-1.,1.,w,dtype=torch.float32,device=device))
    z = torch.as_tensor(depth_tensor, dtype=torch.float32, device=device)
    xyz_old_world = torch.stack((x.flatten(), y.flatten(), z.flatten()), dim=1)

    xyz_old_cam_xy = persp_cam_old.get_full_projection_transform().transform_points(xyz_old_world)[:,0:2]
    xyz_new_cam_xy = persp_cam_new.get_full_projection_transform().transform_points(xyz_old_world)[:,0:2]

    offset_xy = xyz_new_cam_xy - xyz_old_cam_xy
    # affine_grid theta param expects a batch of 2D mats. Each is 2x3 to do rotation+translation.
    identity_2d_batch = torch.tensor([[1.,0.,0.],[0.,1.,0.]], device=device).unsqueeze(0)
    # coords_2d will have shape (N,H,W,2).. which is also what grid_sample needs.
    coords_2d = torch.nn.functional.affine_grid(identity_2d_batch, [1,1,h,w], align_corners=False)
    offset_coords_2d = coords_2d - torch.reshape(offset_xy, (h,w,2)).unsqueeze(0)

    image_tensor = torchvision.transforms.functional.to_tensor(Image.fromarray(prev_img_cv2)).to(device)
    new_image = torch.nn.functional.grid_sample(
        image_tensor.add(1/512 - 0.0001).unsqueeze(0), 
        offset_coords_2d, 
        mode=anim_args.sampling_mode, 
        padding_mode=anim_args.padding_mode, 
        align_corners=False
    )

    # convert back to cv2 style numpy array 0->255 uint8
    result = rearrange(
        new_image.squeeze().clamp(0,1) * 255.0, 
        'c h w -> h w c'
    ).cpu().numpy().astype(np.uint8)
    return result

def generate(args, return_latent=False, return_sample=False, return_c=False):
    seed_everything(args.seed)
    os.makedirs(args.outdir, exist_ok=True)

    if args.sampler == 'plms':
        sampler = PLMSSampler(model)
    else:
        sampler = DDIMSampler(model)

    model_wrap = CompVisDenoiser(model)       
    batch_size = args.n_samples
    prompt = args.prompt
    assert prompt is not None
    data = [batch_size * [prompt]]

    init_latent = None
    if args.init_latent is not None:
        init_latent = args.init_latent
    elif args.init_sample is not None:
        init_latent = model.get_first_stage_encoding(model.encode_first_stage(args.init_sample))
  
    elif args.use_init and args.init_image != None and args.init_image != '':
        init_image = load_img(args.init_image, shape=(args.W, args.H)).to(device)
        init_image = repeat(init_image, '1 ... -> b ...', b=batch_size)
        init_latent = model.get_first_stage_encoding(model.encode_first_stage(init_image))  # move to latent space        

    if not args.use_init and args.strength > 0:
        print("\nNo init image, but strength > 0. This may give you some strange results.\n")

    # Mask functions
    mask = None
    if args.use_mask:
        assert args.mask_file is not None, "use_mask==True: An mask image is required for a mask"
        assert args.use_init, "use_mask==True: use_init is required for a mask"
        assert init_latent is not None, "use_mask==True: An latent init image is required for a mask"

        mask = prepare_mask(args.mask_file, 
                            init_latent.shape, 
                            args.mask_contrast_adjust, 
                            args.mask_brightness_adjust)
        
        mask = mask.to(device)
        mask = repeat(mask, '1 ... -> b ...', b=batch_size)
        
    t_enc = int((1.0-args.strength) * args.steps)

    # Noise schedule for the k-diffusion samplers (used for masking)
    k_sigmas = model_wrap.get_sigmas(args.steps)
    k_sigmas = k_sigmas[len(k_sigmas)-t_enc-1:]

    if args.sampler in ['plms','ddim']:
        sampler.make_schedule(ddim_num_steps=args.steps, ddim_eta=args.ddim_eta, ddim_discretize='fill', verbose=False)

    callback = make_callback(sampler_name=args.sampler,
                            dynamic_threshold=args.dynamic_threshold, 
                            static_threshold=args.static_threshold,
                            mask=mask, 
                            init_latent=init_latent,
                            sigmas=k_sigmas,
                            sampler=sampler,
                            showdiffusion=args.showinterimg,
                            savediffusion=args.saveinterimg,
                            listsaveddiffusion=args.listinterimg,
                            skipsamples=args.skipsamples,
                            countsamples=args.countsamples)    

    results = []
    precision_scope = autocast if args.precision == "autocast" else nullcontext
    with torch.no_grad():
        with precision_scope("cuda"):
            with model.ema_scope():
                for prompts in data:
                    uc = None
                    if args.scale != 1.0:
                        #uc = model.get_learned_conditioning(batch_size * [""])
                        uc = model.get_learned_conditioning([args.negativeprompt])
                    if isinstance(prompts, tuple):
                        prompts = list(prompts)
                    c = model.get_learned_conditioning(prompts)

                    if args.init_c != None:
                        c = args.init_c

                    if args.sampler in ["klms","dpm2","dpm2_ancestral","heun","euler","euler_ancestral"]:
                        samples = sampler_fn(
                            c=c, 
                            uc=uc, 
                            args=args, 
                            model_wrap=model_wrap, 
                            init_latent=init_latent, 
                            t_enc=t_enc, 
                            device=device, 
                            cb=callback)
                    else:
                        # args.sampler == 'plms' or args.sampler == 'ddim':
                        if init_latent is not None and args.strength > 0:
                            z_enc = sampler.stochastic_encode(init_latent, torch.tensor([t_enc]*batch_size).to(device))
                        else:
                            z_enc = torch.randn([args.n_samples, args.C, args.H // args.f, args.W // args.f], device=device)
                        if args.sampler == 'ddim':
                            samples = sampler.decode(z_enc, 
                                                     c, 
                                                     t_enc, 
                                                     unconditional_guidance_scale=args.scale,
                                                     unconditional_conditioning=uc,
                                                     img_callback=callback)
                        elif args.sampler == 'plms': # no "decode" function in plms, so use "sample"
                            shape = [args.C, args.H // args.f, args.W // args.f]
                            samples, _ = sampler.sample(S=args.steps,
                                                            conditioning=c,
                                                            batch_size=args.n_samples,
                                                            shape=shape,
                                                            verbose=False,
                                                            unconditional_guidance_scale=args.scale,
                                                            unconditional_conditioning=uc,
                                                            eta=args.ddim_eta,
                                                            x_T=z_enc,
                                                            img_callback=callback)
                        else:
                            raise Exception(f"Sampler {args.sampler} not recognised.")

                    if return_latent:
                        results.append(samples.clone())

                    x_samples = model.decode_first_stage(samples)
                    if return_sample:
                        results.append(x_samples.clone())

                    x_samples = torch.clamp((x_samples + 1.0) / 2.0, min=0.0, max=1.0)

                    if return_c:
                        results.append(c.clone())

                    for x_sample in x_samples:
                        x_sample = 255. * rearrange(x_sample.cpu().numpy(), 'c h w -> h w c')
                        image = Image.fromarray(x_sample.astype(np.uint8))
                        results.append(image)
    return results

In [ ]:
#@markdown **Select and Load Model**

model_config = "v1-inference.yaml" #@param ["custom","v1-inference.yaml"]
model_checkpoint =  "sd-v1-4.ckpt" #@param ["custom","sd-v1-4-full-ema.ckpt","sd-v1-4.ckpt","sd-v1-3-full-ema.ckpt","sd-v1-3.ckpt","sd-v1-2-full-ema.ckpt","sd-v1-2.ckpt","sd-v1-1-full-ema.ckpt","sd-v1-1.ckpt","robo-diffusion-v1.ckpt"]
custom_config_path = "" #@param {type:"string"}
custom_checkpoint_path = "" #@param {type:"string"}

load_on_run_all = True #@param {type: 'boolean'}
half_precision = True # check
check_sha256 = False #@param {type:"boolean"}

model_map = {
    "sd-v1-4-full-ema.ckpt": {'sha256': '14749efc0ae8ef0329391ad4436feb781b402f4fece4883c7ad8d10556d8a36a'},
    "sd-v1-4.ckpt": {'sha256': 'fe4efff1e174c627256e44ec2991ba279b3816e364b49f9be2abc0b3ff3f8556'},
    "sd-v1-3-full-ema.ckpt": {'sha256': '54632c6e8a36eecae65e36cb0595fab314e1a1545a65209f24fde221a8d4b2ca'},
    "sd-v1-3.ckpt": {'sha256': '2cff93af4dcc07c3e03110205988ff98481e86539c51a8098d4f2236e41f7f2f'},
    "sd-v1-2-full-ema.ckpt": {'sha256': 'bc5086a904d7b9d13d2a7bccf38f089824755be7261c7399d92e555e1e9ac69a'},
    "sd-v1-2.ckpt": {'sha256': '3b87d30facd5bafca1cbed71cfb86648aad75d1c264663c0cc78c7aea8daec0d'},
    "sd-v1-1-full-ema.ckpt": {'sha256': 'efdeb5dc418a025d9a8cc0a8617e106c69044bc2925abecc8a254b2910d69829'},
    "sd-v1-1.ckpt": {'sha256': '86cd1d3ccb044d7ba8db743d717c9bac603c4043508ad2571383f954390f3cea'},
    "robo-diffusion-v1.ckpt": {'sha256': '244dbe0dcb55c761bde9c2ac0e9b46cc9705ebfe5f1f3a7cc46251573ea14e16'}

# ================================================================================================ #
# NOTE: for robo-diffusion to work you must have the words 'nousr' and 'robot' next to each other  #
# ================================================================================================ #
# Example: 'A photo of a nousr robot'                                                              #
# Example: 'A water color painting of a nousr robot, in the style of studio ghibli'                #
# ================================================================================================ #

#prompts = [
#    "A water color painting of a nousr robot, in the style of studio ghibli",
#]

    
}

# config path
ckpt_config_path = custom_config_path if model_config == "custom" else os.path.join(models_path, model_config)
if os.path.exists(ckpt_config_path):
    print(f"{ckpt_config_path} exists")
else:
    ckpt_config_path = "./stable-diffusion/configs/stable-diffusion/v1-inference.yaml"
print(f"Using config: {ckpt_config_path}")

# checkpoint path or download
ckpt_path = custom_checkpoint_path if model_checkpoint == "custom" else os.path.join(models_path, model_checkpoint)
ckpt_valid = True
if os.path.exists(ckpt_path):
    print(f"{ckpt_path} exists")
else:
    print(f"Please download model checkpoint and place in {os.path.join(models_path, model_checkpoint)}")
    ckpt_valid = False

if check_sha256 and model_checkpoint != "custom" and ckpt_valid:
    import hashlib
    print("\n...checking sha256")
    with open(ckpt_path, "rb") as f:
        bytes = f.read() 
        hash = hashlib.sha256(bytes).hexdigest()
        del bytes
    if model_map[model_checkpoint]["sha256"] == hash:
        print("hash is correct\n")
    else:
        print("hash in not correct\n")
        ckpt_valid = False
       
if ckpt_valid:
    print(f"Using ckpt: {ckpt_path}")

def load_model_from_config(config, ckpt, verbose=False, device='cuda', half_precision=True):
    map_location = "cuda" #@param ["cpu", "cuda"]
    print(f"Loading model from {ckpt}")
    pl_sd = torch.load(ckpt, map_location=map_location)
    if "global_step" in pl_sd:
        print(f"Global Step: {pl_sd['global_step']}")
    sd = pl_sd["state_dict"]
    model = instantiate_from_config(config.model)
    m, u = model.load_state_dict(sd, strict=False)
    if len(m) > 0 and verbose:
        print("missing keys:")
        print(m)
    if len(u) > 0 and verbose:
        print("unexpected keys:")
        print(u)

    if half_precision:
        model = model.half().to(device)
    else:
        model = model.to(device)
    model.eval()
    return model

if load_on_run_all and ckpt_valid:
    local_config = OmegaConf.load(f"{ckpt_config_path}")
    model = load_model_from_config(local_config, f"{ckpt_path}",half_precision=half_precision)
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    model = model.to(device)

In [ ]:
%%capture
#@markdown **Auxiliar Instalations**
Execute=True #@param {type:"boolean"}
if Execute:
  # Clone Real-ESRGAN and enter the Real-ESRGAN
  !git clone https://github.com/xinntao/Real-ESRGAN.git
  %cd Real-ESRGAN
  # Set up the environment
  !pip install basicsr
  !pip install facexlib
  !pip install gfpgan
  !pip install -r requirements.txt
  !python setup.py develop
  #warning import duplicate?
  import matplotlib.pyplot as plt
  def display2(img1, img2):
    fig = plt.figure(figsize=(25, 10))
    ax1 = fig.add_subplot(1, 2, 1) 
    plt.title('Input', fontsize=16)
    ax1.axis('off')
    ax2 = fig.add_subplot(1, 2, 2)
    plt.title('CodeFormer', fontsize=16)
    ax2.axis('off')
    ax1.imshow(img1)
    ax2.imshow(img2)
  def imread(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

  #################################




In [ ]:
#@title Widgets imports
# Imports for JupyterLite
try:
    import piplite
    await piplite.install(['ipywidgets'])
except ImportError:
    pass

import ipywidgets as widgets




In [ ]:
#@markdown **Auxiliar functions**

from PIL.PngImagePlugin import PngInfo
from os import remove

try:
  from base64 import b64encode
  from base64 import b64decode
  from IPython.display import HTML
  from google.colab.output import eval_js
  from Crypto.Cipher import AES
  from Crypto.Util.Padding import pad
  from Crypto.Util.Padding import unpad
  from google.colab.output import eval_js
except:
  print('import error')



def encript(data):
  iv=   b'\xaf\x00\n\x96\xf5\x9f\xde\x02\xb8 \x88w\xf2\x83\x99\x9e'
  key = b'&\xbf\x18\x1f\x0b\x12\x06\x0c\x0e\xc6m\xd6\xaf0uZ\xfdb\x89\xe7o\xd8\xff=D\xde\x86H\x03j\xc2,'
  data = bytes(data, 'utf-8')
  cipher = AES.new(key, AES.MODE_CBC, iv)

  ct_bytes = cipher.encrypt(pad(data, AES.block_size))
  iv = b64encode(iv).decode('utf-8')
  ct = b64encode(ct_bytes).decode('utf-8')
  return ct
  
 


def decript(data):
  iv=   b'\xaf\x00\n\x96\xf5\x9f\xde\x02\xb8 \x88w\xf2\x83\x99\x9e'
  key = b'&\xbf\x18\x1f\x0b\x12\x06\x0c\x0e\xc6m\xd6\xaf0uZ\xfdb\x89\xe7o\xd8\xff=D\xde\x86H\x03j\xc2,'
  try:
      ct = b64decode(data)
      cipher = AES.new(key, AES.MODE_CBC, iv)
      pt = unpad(cipher.decrypt(ct), AES.block_size)
      return pt.decode("utf-8")
  except: 
      print("Incorrect decryption")

def cbool(sbool):
  if sbool == "True" or sbool=="true": return True 
  elif sbool=="False" or sbool=="false": return False
  else: return "null"


def ParamToMeta(filename, args):
    filepath=args.outdir
    OriginFilePath=os.path.join(filepath,filename + ".png")
    TargetFilePath=os.path.join(filepath, filename + "_meta.png")
    targetImage = Image.open(OriginFilePath)
    metadata = PngInfo()
    metadata.add_text("v",'0.1')
    metadata.add_text("enc",'1')
    metadata.add_text("W", str(args.W))
    metadata.add_text("H", str(args.H))
    metadata.add_text("sampler", args.sampler)
    metadata.add_text("steps", str(args.steps))
    metadata.add_text("scale", str(args.scale))
    metadata.add_text("ddim_eta", str(args.ddim_eta))
    metadata.add_text("dynamic_threshold", "null")
    metadata.add_text("static_threshold", "null")
    metadata.add_text("save_samples", str(args.save_samples))
    metadata.add_text("save_settings", str(args.save_settings))
    metadata.add_text("display_samples", str(args.display_samples))
    metadata.add_text("n_batch", str(args.n_batch))
    metadata.add_text("batch_name", args.batch_name)
    metadata.add_text("seed_behavior", args.seed_behavior)
    metadata.add_text("make_grid", str(args.make_grid))
    metadata.add_text("grid_rows", str(args.grid_rows))
    metadata.add_text("outdir", args.outdir)
    metadata.add_text("use_init", str(args.use_init))
    metadata.add_text("strength", str(args.strength))
    metadata.add_text("use_mask", str(args.use_mask))
    metadata.add_text("mask_file", args.mask_file)
    metadata.add_text("invert_mask", str(args.invert_mask))
    metadata.add_text("mask_brightness_adjust", str(args.mask_brightness_adjust))
    metadata.add_text("mask_contrast_adjust", str(args.mask_contrast_adjust))
    metadata.add_text("n_samples", str(args.n_samples))
    metadata.add_text("precision", args.precision)
    metadata.add_text("C", str(args.C))
    metadata.add_text("f", str(args.f))
    metadata.add_text("timestring", args.timestring)
    metadata.add_text("init_latent", "null")
    metadata.add_text("init_sample", "null")
    metadata.add_text("init_c", "null")
    metadata.add_text("prompts", "")
    metadata.add_text("negativeprompt",encript(args.negativeprompt))
    metadata.add_text("seed", encript(str(args.seed)))
    metadata.add_text("filename_format", encript(args.filename_format))
    metadata.add_text("init_image", encript(args.init_image))
    metadata.add_text("prompt", encript(args.prompt))
    targetImage.save(TargetFilePath, pnginfo=metadata)
    print(targetImage.info)
    targetImage.close()
    remove(OriginFilePath)
    return TargetFilePath


def MetaToParam(vdict, args):
  try:
    vdict.get('v')
    args.ww.value= int(vdict.get('W'))
    args.wh.value= np.int_(vdict.get('H'))
    args.wsampler.value= (vdict.get('sampler'))
    args.wsteps.value= int(vdict.get('steps'))
    args.wscale.value= float(vdict.get('scale'))
    args.wddim_eta.value= float(vdict.get('ddim_eta'))
    args.wsave_samples.value= cbool(vdict.get('save_samples'))
    args.wdisplay_samples.value= cbool(vdict.get('display_samples'))
    #args.wn_batch.value= int(vdict.get('n_batch'))
    args.wn_batch.value= 1
    args.wbatch_name.value= (vdict.get('batch_name'))
    args.wseed_behavior.value= (vdict.get('seed_behavior'))
    args.wmake_grid.value= cbool(vdict.get('make_grid'))
    args.wgrid_rows.value= int(vdict.get('grid_rows'))
    args.wuse_init.value= cbool(vdict.get('use_init'))
    args.wstrength.value= float(vdict.get('strength'))
    args.wuse_mask.value= cbool(vdict.get('use_mask'))
    args.wmask_file.value= (vdict.get('mask_file'))
    args.winvert_mask.value= cbool(vdict.get('invert_mask'))
    args.wmask_brightness_adjust.value= float(vdict.get('mask_brightness_adjust'))
    args.wmask_contrast_adjust.value= float(vdict.get('mask_contrast_adjust'))
    args.wprompt.value= decript(vdict.get('prompt'))
    args.wseed.value= int(decript(vdict.get('seed')))
    args.wnegativeprompt.value=decript(vdict.get('negativeprompt'))
    args.wfilename_format.value= decript(vdict.get('filename_format'))
    args.winit_image.value= decript(vdict.get('init_image'))
  except:
    try:
      args.ww.value= int(vdict.get('W'))
      args.wh.value= np.int_(vdict.get('H'))
      args.wsampler.value= (vdict.get('sampler'))
      args.wsteps.value= int(vdict.get('steps'))
      args.wscale.value= float(vdict.get('scale'))
      args.wddim_eta.value= float(vdict.get('ddim_eta'))
      args.wsave_samples.value= cbool(vdict.get('save_samples'))
      args.wdisplay_samples.value= cbool(vdict.get('display_samples'))
      #args.wn_batch.value= int(vdict.get('n_batch'))
      args.wn_batch.value= 1
      args.wbatch_name.value= (vdict.get('batch_name'))
      args.wseed_behavior.value= (vdict.get('seed_behavior'))
      args.wmake_grid.value= cbool(vdict.get('make_grid'))
      args.wgrid_rows.value= int(vdict.get('grid_rows'))
      args.wuse_init.value= cbool(vdict.get('use_init'))
      args.wstrength.value= float(vdict.get('strength'))
      args.wuse_mask.value= cbool(vdict.get('use_mask'))
      args.wmask_file.value= (vdict.get('mask_file'))
      args.winvert_mask.value= cbool(vdict.get('invert_mask'))
      args.wmask_brightness_adjust.value= float(vdict.get('mask_brightness_adjust'))
      args.wmask_contrast_adjust.value= float(vdict.get('mask_contrast_adjust'))
      args.wprompt.value= (vdict.get('prompt'))
      args.wseed.value= int(vdict.get('seed'))
      args.wnegativeprompt.value=(vdict.get('negativeprompt'))
      args.wfilename_format.value= (vdict.get('filename_format'))
      args.winit_image.value= (vdict.get('init_image'))
    except:
      print("Metadata Not found, parameteres as default")

canvas_html = """
  
<img src=%s id="imgsource" width="512" height="512" hidden></img>


<canvas id="canvas1" width=%d height=%d style="border: 1px solid black"></canvas>
<canvas id="canvas2" width=%d height=%d style="border: 1px solid black" hidden></canvas>
<button type="button" id="exit">Click Me!</button>

<script>

var canvas = document.getElementById('canvas1')
var canvas2= document.getElementById('canvas2')
var chica= document.getElementById('imgsource')
var ctx = canvas.getContext('2d')
var ctx2= canvas2.getContext('2d')
ctx2.fillStyle = 'white';
ctx2.fillRect( 0, 0, canvas2.width, canvas2.height)
ctx.lineWidth = %d
ctx2.lineWidth = ctx.lineWidth
ctx.drawImage(chica, 0, 0, 512, 512);



var button = document.getElementById("exit")
var mouse = {x: 0, y: 0}
canvas.addEventListener('mousemove', function(e) {
  mouse.x = e.pageX - this.offsetLeft
  mouse.y = e.pageY - this.offsetTop
})
canvas.onmousedown = ()=>{
  ctx.beginPath()
  ctx.moveTo(mouse.x, mouse.y)
  canvas.addEventListener('mousemove', onPaint)

  ctx2.beginPath()
  ctx2.moveTo(mouse.x, mouse.y)
  
}
canvas.onmouseup = ()=>{
  canvas.removeEventListener('mousemove', onPaint)
}
var onPaint = ()=>{
  ctx.lineTo(mouse.x, mouse.y)
  ctx.stroke()

  ctx2.lineTo(mouse.x, mouse.y)
  ctx2.stroke()
}
var data = new Promise(resolve=>{
 button.onclick = ()=>{
    canvas.parentNode.removeChild(canvas);
    chica.parentNode.removeChild(chica)
    button.parentNode.removeChild(button)
    resolve(canvas2.toDataURL('image/png'))
  }
})
</script>
"""

def buildcanvas(w, h, line_width, imgin):
 
 png64=pngtobase64(imgin)
 return  canvas_html % (png64, w, h, w, h, line_width)
  


def cpaint(htmlcanvas, imageout):
  display.display(HTML(htmlcanvas))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  with open(imageout, 'wb') as f:
    f.write(binary)
  return len(binary) 

def pngtobase64(filepath):
    encoded = (base64.b64encode(open(filepath, "rb").read())).decode('ascii')
    return 'data:image/png;base64,{}'.format(encoded)



  


In [ ]:
#@markdown **Param Functions**



def InitParams():

 
  wretro=False
  wretro=widgets.Label(value="False")
  #####---ddim_eta----

  wddim_eta=widgets.FloatSlider(
      value=0.75,
      min=0,
      max=1,
      step=0.05,
      description='ddim_eta',
      disabled=False,
      continuous_update=False,
      orientation='horizontal',
      readout=True,
      readout_format='.2f',
  )

  display.display(wddim_eta)

  #####---save_samples----

  wsave_samples=widgets.Checkbox(
      value=True,
      description='Save Samples',
      disabled=False,
      indent=True
  )

  display.display(wsave_samples)

  #####---save_settings----

  wsave_settings=widgets.Checkbox(
      value=False,
      description='Save Settings',
      disabled=False,
      indent=True
  )

  display.display(wsave_settings)

  #####---display_samples----

  wdisplay_samples=widgets.Checkbox(
      value=True,
      description='display_samples',
      disabled=False,
      indent=True
  )

  display.display(wdisplay_samples)



  

  wfilename_format=widgets.Dropdown(
      options=["{timestring}_{index}_{seed}.png","{timestring}_{index}_{prompt}.png","{timestring}_{index}_{steps}_{seed}.png"],
      value='{timestring}_{index}_{seed}.png',
      description='filename format',
      disabled=False,
  )


  display.display(wfilename_format)

  #####---seed_behavior----

  wseed_behavior=widgets.Dropdown(
      options=["iter","fixed","random"],
      value='iter',
      description='seed behavior',
      disabled=False,
  )

  display.display(wseed_behavior)

  #####---make_grid----

  wmake_grid=widgets.Checkbox(
      value=False,
      description='make grid',
      disabled=False,
      indent=True
  )

  display.display(wmake_grid)

  #####---grid_rows----

  wgrid_rows=widgets.IntText(
      value=2,
      description='grid rows',
      disabled=False
  )

  display.display(wgrid_rows)

  #####---use_init----

  wuse_init=widgets.Checkbox(
      value=False,
      description='use init',
      disabled=False,
      indent=True
  )

  display.display(wuse_init)

  

  #####---init_image----

  winit_image=widgets.Text(
      value='',
      placeholder='Path from Image to ImgtoImg',
      description='init image',
      disabled=False   
  )

  display.display(winit_image)

  #####---use_mask----

  wuse_mask=widgets.Checkbox(
      value=False,
      description='use mask',
      disabled=False,
      indent=True
  )

  display.display(wuse_mask)

  #####---mask_file----

  wmask_file=widgets.Text(
      value='',
      placeholder='Path from mask to ImgtoImg',
      description='mask file',
      disabled=False   
  )

  display.display(wmask_file)

  #####---invert_mask----

  winvert_mask=widgets.Checkbox(
      value=False,
      description='invert mask',
      disabled=False,
      indent=True
  )

  display.display(winvert_mask)

  #####---mask_brightness_adjust----

  wmask_brightness_adjust=widgets.FloatSlider(
      value=1,
      min=0,
      max=1,
      step=0.05,
      description='mask brightness',
      disabled=False,
      continuous_update=False,
      orientation='horizontal',
      readout=True,
      readout_format='.2f',
  )

  display.display(wmask_brightness_adjust)

  #####---mask_contrast_adjust----

  wmask_contrast_adjust=widgets.FloatSlider(
      value=1,
      min=0,
      max=1,
      step=0.05,
      description='mask contrast',
      disabled=False,
      continuous_update=False,
      orientation='horizontal',
      readout=True,
      readout_format='.2f',
  )

  display.display(wmask_contrast_adjust)

 

  #####---wupscale----

  from ipywidgets import interact
  wupscale=widgets.ToggleButtons(
    options=['0x', '1x', '2x', '4x'],
    description='ERSGAN:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or '',
    layout=widgets.Layout(width='250px'),
    tooltips=['No upscale','Restoration without upscale', '2x', '4x'],

)
  
  def wupscale_listener(sender):
    if wupscale.value=='0x':
      wfacerestore.value='Disabled'
      wfacerestore.disabled=True
    else:
       wfacerestore.disabled=False
    return

  wupscale.observe(wupscale_listener, 'value')

  ##@interact
  #def clicked(a=wupscale()):
  #  print(a)

  

  
  #####---wfacerestore----

  wfacerestore=widgets.ToggleButtons(
    options=['Disabled', 'Face Restoration'],
    description='',
    disabled=True,
    button_style='', # 'success', 'info', 'warning', 'danger' or '',
    layout=widgets.Layout(width='200px'),
    tooltips=['No Restoration','Face restoration']

)

  wshowups=widgets.HBox([wupscale, wfacerestore])
  display.display(wshowups)

   #####---confgimg----

  wconfgimg=widgets.Text(
      value='',
      placeholder='Path from Image Config',
      description='Config Image',
      disabled=False   
  )

  display.display(wconfgimg)

  wfile=open("/content/drive/MyDrive/AI/auxiliar/empty.png", "rb")
  wimgempty= wfile.read()

   #####---showinterimg----

  wshowinterimg=widgets.Checkbox(
      value=False,
      description='Show Sampling',
      disabled=False,
      indent=True
  )

  display.display(wshowinterimg)

  #####---saveinterimg----

  wsaveinterimg=widgets.Checkbox(
      value=False,
      description='Save Sampling',
      disabled=False,
      indent=True
  )

 
  display.display(wsaveinterimg)

  #####---skipsamples----

  wskipsamples=widgets.IntSlider(
      value=10,
      min=1,
      max=25,
      step=1,
      description='Skip Samples',
      disabled=False,
      continuous_update=False,
      orientation='horizontal',
      readout=True,
      readout_format='d'
  )

  display.display(wskipsamples)

  def LoadImgConfig(vbutton):
  #789
   imgbytes=(wupload.value.get(wupload.metadata[0].get('name'))).get('content')
   wimgshowconf.value=imgbytes
   wimgresult.value=imgbytes
   smetadata=Image.open(io.BytesIO(imgbytes)).info
   if bool(smetadata):
      MetaToParam(smetadata,parenv)
   else:
      print("Image without configuration parameters!")
  
  

 #####---wupload, wloadmeta---

  wupload=widgets.FileUpload(
  accept='.png',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
  multiple=False  # True to accept multiple files upload else False 
  )

  wupload.observe(LoadImgConfig, 'value')


  tfile=open("/content/drive/MyDrive/AI/auxiliar/empty.png", "rb")
  timage=image = tfile.read()

  wimgshowconf=widgets.Image(
    value=timage,
    format='png',
    width=32,
    height=32,
  )

  ui = widgets.HBox([wupload, wimgshowconf])  


  display.display(ui)

  def wstartevent(sender):
 
    #sender.disabled=True
    sender.description='Processing...'
    sender.button_style='warning'
    initinference(parenv)
    #sender.disabled=False
    sender.description='START!'
    sender.button_style='info'

#####---prompt----

  wprompt=widgets.Textarea(
      value='gorgeous woman with a long hair waving with wind, intricate, elegant, sharp focus, illustration, highly detailed, digital painting, concept art, matte, art by WLOP and Artgerm and Greg Rutkowski and Alphonse Mucha, masterpiece',
      placeholder='Type something',
      description='Prompt:',
      disabled=False,
      layout=widgets.Layout(height='200px'),
  )
  

  #####---wnegativeprompt----

  wnegativeprompt=widgets.Textarea(
      value='',
      placeholder='Type something',
      description='Negative:',
      disabled=False,
      layout=widgets.Layout(height='200px'),
  )
   
  uipn = widgets.HBox([wprompt, wnegativeprompt])
  display.display(uipn)  

 #####---H----

  wh=widgets.IntText(
      value=512,
      description='H',
      disabled=False
  )

  #display.display(wh)

 
  #####---W----

  ww=widgets.IntText(
      value=512,
      description='W',
      disabled=False
  )

  #display.display(ww)

  uiwwwh = widgets.HBox([wh, ww])  
  display.display(uiwwwh)

  #####---n_batch----

  wn_batch=widgets.IntText(
      value=1,
      description='n batch',
      disabled=False
  )

  #display.display(wn_batch)  

  #####---batch_name----

  wbatch_name=widgets.Text(
      value='',
      placeholder='Name the Batch folder',
      description='batch name',
      disabled=False   
  )

  #display.display(wbatch_name)
  ui_nbatch_batch_name = widgets.HBox([wn_batch, wbatch_name])  
  display.display(ui_nbatch_batch_name)

  #####---filename_format----

 #####---seed----

  wseed=widgets.IntText(
      value=-1,
      description='seed',
      disabled=False
  )

  #display.display(wseed)

  #####---sampler----

  wsampler=widgets.Dropdown(
      options=["klms","dpm2","dpm2_ancestral","heun","euler","euler_ancestral","plms", "ddim"],
      value='klms',
      description='Sampler',
      disabled=False,
  )


  #display.display(wsampler)

  ui_seed_sampler = widgets.HBox([wseed, wsampler])  
  display.display(ui_seed_sampler)  

#####---steps----

  def wsteps_listener(sender):
    wimgtrans.value=1
    wimgtrans.max=int(wsteps.value*(1-wstrength.value))
    
    

  wsteps=widgets.IntSlider(
      value=50,
      min=0,
      max=400,
      step=5,
      description='Steps',
      disabled=False,
      continuous_update=False,
      orientation='horizontal',
      readout=True,
      readout_format='d'
  )

  wsteps.observe(wsteps_listener, 'value')

  #display.display(wsteps)

     #####---scale----

  wscale=widgets.FloatSlider(
      value=7.5,
      min=0,
      max=50.0,
      step=0.5,
      description='Scale',
      disabled=False,
      continuous_update=False,
      orientation='horizontal',
      readout=True,
      readout_format='.1f',
  )

  #display.display(wscale)

  ui_steeps_scale = widgets.HBox([wsteps, wscale])  
  display.display(ui_steeps_scale)  

#####---strength----

  wstrength=widgets.FloatSlider(
      value=0,
      min=0,
      max=1,
      step=0.05,
      description='strength',
      disabled=False,
      continuous_update=False,
      orientation='horizontal',
      readout=True,
      readout_format='.2f',
  )

  wstrength.observe(wsteps_listener, 'value')
  display.display(wstrength)

#####---wstart---
  def showpaint(sender):
    vcanvas=buildcanvas(512,512,10,'/content/temporal/1.png')
    cpaint(vcanvas,'/content/temporal/mask.png')

  wstart = widgets.Button(
  description='START',
  disabled=False,
  button_style='info', # 'success', 'info', 'warning', 'danger' or ''
  tooltip='Click me',
  icon='check' # (FontAwesome names without the `fa-` prefix)
  )

  wstart.on_click(wstartevent)
  #display.display(wstart)

  #####---wresulttoinit---

  def ResultToInit(sender):
    img = Image.open(io.BytesIO(wimgresult.value),mode='r')
    filename="temporal.png"
    OriginFilePath=os.path.join(temporal_path,filename)
    img.save(OriginFilePath, format='PNG')
    img.close()
    wuse_init.value=True
    winit_image.value=OriginFilePath
    sender.button_style='warning'
    wretro.value="True"


  def ResultToInitoff(sender):
    
    wuse_init.value=False
    winit_image.value=""
    wstrength.value=0
    wresulttoinit.button_style='info'
    wretro.value="False"
    

  wresulttoinit = widgets.Button(
  description='RETRO ON',
  disabled=False,
  button_style='info', # 'success', 'info', 'warning', 'danger' or ''
  tooltip='Click me',
  icon='check' # (FontAwesome names without the `fa-` prefix)
  )

  wresulttoinit.on_click(ResultToInit)
  #display.display(wresulttoinit)

  wresulttoinitoff = widgets.Button(
  description='RETRO OFF',
  disabled=False,
  button_style='info', # 'success', 'info', 'warning', 'danger' or ''
  tooltip='Click me',
  icon='check' # (FontAwesome names without the `fa-` prefix)
  )

  wresulttoinitoff.on_click(ResultToInitoff)
 # display.display(wresulttoinitoff)

 #####---wshowpaint---

  wshowpaint = widgets.Button(
  description='MASK',
  disabled=False,
  button_style='info', # 'success', 'info', 'warning', 'danger' or ''
  tooltip='Image mask',
  icon='check' # (FontAwesome names without the `fa-` prefix)
  )

  wshowpaint.on_click(showpaint)

  ui_start_wresulttoinit_wresultoinitoff = widgets.HBox([wstart, wresulttoinit, wresulttoinitoff, wshowpaint])  
  display.display(ui_start_wresulttoinit_wresultoinitoff) 

  #####---wimgtrans----

  def wimgtrans_listener(sender):
    internamefile=os.path.join(temporal_path, str(wimgtrans.value) + ".png")
    print(internamefile)   
    img = open(internamefile,'rb')
    wimgresult.value=img.read()
   
   
 


  wimgtrans=widgets.IntSlider(
    value=1,
    min=1,
    max=50,
    step=1,
    description='Intermedie:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
  
  wimgtrans.observe(wimgtrans_listener, 'value')
  display.display(wimgtrans)


  return locals()

def BuildErsganParams(upsizeopt, faceopt, inputimage, outputpath):
  
  face=''
  if faceopt:
    face='--face_enhance'
  origenimg=inputimage
  targetpath=outputpath
  
  return (f'-n RealESRGAN_x4plus -i {origenimg} -o {targetpath} --outscale {upsizeopt}  {face} --suffix ersgan ')



   # if it is out of memory, try to use the `--tile` option
  # We upsample the image with the scale factor X3.5

  # par='-n RealESRGAN_x4plus -i upload -o results --outscale 4 --fp32 --face_enhance'
  # !python inference_realesrgan.py $par
  # Arguments
  # -n, --model_name: Model names
  # -i, --input: input folder or image
  # --outscale: Output scale, can be arbitrary scale factore. 
  #[-h] [-i INPUT] [-n MODEL_NAME] [-o OUTPUT]
  #                               [-dn DENOISE_STRENGTH] [-s OUTSCALE]
  #                               [--model_path MODEL_PATH] [--suffix SUFFIX]
  #                               [-t TILE] [--tile_pad TILE_PAD]
  #                              [--pre_pad PRE_PAD] [--face_enhance] [--fp32]
  #                               [--alpha_upsampler ALPHA_UPSAMPLER] [--ext EXT]
  #                              [-g GPU_ID]
    


def DeforumArgs(args):
      

      class counter:
       
        def __init__(self):
          self.x=0
        def inc(self):
           self.x=self.x+1
        def reset(self):
           self.x=0
        def value(self):
           return self.x
        

      BatchPrompts=False #@param {type:"boolean"}

      bprompts=["a beautiful forest by Asher Brown Durand, trending on Artstation", #the first prompt I want
            "a beautiful portrait of a woman by Artgerm, trending on Artstation", #the second prompt I want
              #"the third prompt I don't want it I commented it with an",
            ]

      retro=cbool(args.wretro.value)
      showinterimg=args.wshowinterimg.value
      saveinterimg=args.wsaveinterimg.value
      listinterimg=[]
      skipsamples=args.wskipsamples.value
      ##cuidado multiples referencias
      countsamples=counter()
      countsamples.reset()
      incr_batch=False;
      incr_every=0;
      upscale_scale=args.wupscale.value[:1]
      face_restore=False;
      if args.wfacerestore.value!='Disabled':
        face_restore=True;
      prompt=args.wprompt.value
      negativeprompt=args.wnegativeprompt.value
      prompts=[]
      if BatchPrompts==False:
        prompts.append(prompt)
      else:
        prompts=bprompts

      W = args.ww.value
      H = args.wh.value
      W, H = map(lambda x: x - x % 64, (W, H))  # resize to integer multiple of 64

     
      seed = args.wseed.value
      sampler = parenv.wsampler.value 
      steps = args.wsteps.value
      scale = args.wscale.value
      ddim_eta = args.wddim_eta.value
      dynamic_threshold = None
      static_threshold = None   

      save_samples = args.wsave_samples.value
      save_settings = args.wsave_settings.value
      display_samples = args.wdisplay_samples.value
      Save_Conf_in_Image= True

     
      n_batch = args.wn_batch.value
      batch_name = args.wbatch_name.value
      filename_format =args.wfilename_format.value
      if incr_batch:
        seed_behavior='fixed'
      else: 
        seed_behavior = args.wseed_behavior.value
      make_grid = args.wmake_grid.value
      grid_rows = args.wgrid_rows.value
      outdir = get_output_folder(output_path, batch_name)
      
     
      use_init = args.wuse_init.value
      strength = args.wstrength.value
      init_image = args.winit_image.value
      # Whiter areas of the mask are areas that change more
      use_mask = args.wuse_mask.value
      mask_file = args.wmask_file.value
      invert_mask = args.winvert_mask.value
      # Adjust mask image, 1.0 is no adjustment. Should be positive numbers.
      mask_brightness_adjust = args.wmask_brightness_adjust.value
      mask_contrast_adjust = args.wmask_contrast_adjust.value

      n_samples = 1 # doesnt do anything
      precision = 'autocast' 
      C = 4
      f = 8

      prompt = ""
      timestring = ""
      init_latent = None
      init_sample = None
      init_c = None

      return locals()



def next_seed(args):
      if args.seed_behavior == 'iter':
          args.seed += 1
      elif args.seed_behavior == 'fixed':
          pass # always keep seed the same
      else:
          args.seed = random.randint(0, 2**32)
      return args.seed

def render_image_batch(args, params):
     
      args.prompts = {k: f"{v:05d}" for v, k in enumerate(args.prompts)}
      
      # create output folder for the batch
      os.makedirs(args.outdir, exist_ok=True)
      if args.save_settings or args.save_samples:
          print(f"Saving to {os.path.join(args.outdir, args.timestring)}_*")

      # save settings for the batch
      if args.save_settings:
          filename = os.path.join(args.outdir, f"{args.timestring}_settings.txt")
          with open(filename, "w+", encoding="utf-8") as f:
              json.dump(dict(args.__dict__), f, ensure_ascii=False, indent=4)

      index = 0
      
      # function for init image batching
      init_array = []
      if args.use_init:
          if args.init_image == "":
              raise FileNotFoundError("No path was given for init_image")
          if args.init_image.startswith('http://') or args.init_image.startswith('https://'):
              init_array.append(args.init_image)
          elif not os.path.isfile(args.init_image):
              if args.init_image[-1] != "/": # avoids path error by adding / to end if not there
                  args.init_image += "/" 
              for image in sorted(os.listdir(args.init_image)): # iterates dir and appends images to init_array
                  if image.split(".")[-1] in ("png", "jpg", "jpeg"):
                      init_array.append(args.init_image + image)
          else:
              init_array.append(args.init_image)
      else:
          init_array = [""]

      # when doing large batches don't flood browser with images
      clear_between_batches = args.n_batch >= 32

      for iprompt, prompt in enumerate(args.prompts):  
          args.prompt = prompt
          print(f"Prompt {iprompt+1} of {len(args.prompts)}")
          print(f"{args.prompt}")
        
          all_images = []

          for batch_index in range(args.n_batch):
              # if clear_between_batches and batch_index % 32 == 0: 
              #     display.clear_output(wait=True)            
              print(f"Batch {batch_index+1} of {args.n_batch}")
              
              for image in init_array: # iterates the init images
                  args.init_image = image
                  print(args.steps)
                  results = generate(args)
                  for image in results:
                      if args.make_grid:
                          all_images.append(T.functional.pil_to_tensor(image))
                      if args.save_samples:
                          if args.filename_format == "{timestring}_{index}_{prompt}.png":
                              partialfilename = f"{args.timestring}_{index:05}_{sanitize(prompt)[:160]}"
                          elif args.filename_format == "{timestring}_{index}_{steps}_{seed}.png":
                              partialfilename = f"{args.timestring}_{index:05}_{args.steps}_{args.seed}"
                          else:
                              partialfilename = f"{args.timestring}_{index:05}_{args.seed}"
                          filename=partialfilename + ".png"
                          image.save(os.path.join(args.outdir,filename))
                          if args.Save_Conf_in_Image:
                            LastFile=ParamToMeta(partialfilename, args)
                            if args.upscale_scale!='0':
                              print(args.face_restore)
                              bes= BuildErsganParams(args.upscale_scale, args.face_restore, LastFile, args.outdir)
                              print(bes)
                              !python inference_realesrgan.py $bes
                              imgupsname=partialfilename + '_meta_ersgan.png'
                              img2=open(os.path.join(args.outdir,imgupsname),'rb')
                              wimgresultretouch.value=img2.read()
                              img2.close

                      if args.display_samples:
                          buf = io.BytesIO()
                          image.save(buf, format='PNG')
                          byte_im = buf.getvalue()
                          wimgresult.value=byte_im
                          buf.flush
                          buf.close
                          image.close
                          #789
                      vlen=len(args.listinterimg)
                      print(vlen)
                      if vlen>0:
                        print(args.retro)
                        if args.retro==False:
                          pathdiff=os.path.join(args.outdir,partialfilename) 
                          os.makedirs(pathdiff, exist_ok=True)
                          for x in range(vlen):
                            #print((vlen-1)-x)
                            partialdiffpath=os.path.join(pathdiff, partialfilename +  "_diff_" + str((vlen-1)-x))
                            #print(partialdiffpath)
                            args.listinterimg[(vlen-1)-x].save(partialdiffpath + ".png")
                            if args.Save_Conf_in_Image:
                              ParamToMeta(partialdiffpath, args)
                            args.listinterimg[(vlen-1)-x].close 
                            del args.listinterimg[((vlen-1)-x)] 
                        else:
                          for x in range(vlen):
                            #000
                            internamefile=os.path.join(temporal_path, str(((vlen-1)-x)+1) + ".png")   
                            args.listinterimg[(vlen-1)-x].save(internamefile)
                            args.listinterimg[(vlen-1)-x].close 
                            del args.listinterimg[((vlen-1)-x)] 


                      index += 1
                  args.seed = next_seed(args)
                  if args.incr_batch:
                    args.steps+=args.incr_every

          #print(len(all_images))
          if args.make_grid:
              grid = make_grid(all_images, nrow=int(len(all_images)/args.grid_rows))
              grid = rearrange(grid, 'c h w -> h w c').cpu().numpy()
              filename = f"{args.timestring}_{iprompt:05d}_grid_{args.seed}.png"
              grid_image = Image.fromarray(grid.astype(np.uint8))
              grid_image.save(os.path.join(args.outdir, filename))
              #display.clear_output(wait=True)            
              #display.display(grid_image)
              buf = io.BytesIO()
              grid_image.save(buf, format='PNG')
              byte_im = buf.getvalue()
              wimgresult.value=byte_im



def initinference (params):
 
  args = SimpleNamespace(**DeforumArgs(params))
  args.timestring = time.strftime('%Y%m%d%H%M%S')
  args.strength = max(0.0, min(1.0, args.strength))

  if args.seed == -1:
      args.seed = random.randint(0, 2**32)
  if not args.use_init:
      args.init_image = None
  if args.sampler == 'plms' and (args.use_init):
      print(f"Init images are n't supported with PLMS yet, switching to KLMS")
      args.sampler = 'klms'
  if args.sampler != 'ddim':
      args.ddim_eta = 0
      
  # dispatch to appropriate renderer

  wimgresult.value=parenv.wimgempty
  # print("!!!!!!!")
  # print(len(args.listinterimg))
  # print("!!!!!!!")
  image=render_image_batch(args,params) 
  ##ERSGAN post processing

  
  # REparams=BuildErsganParams()
  # par='-n RealESRGAN_x4plus -i upload -o results --outscale 4 --fp32 --face_enhance'
  # # !python inference_realesrgan.py $par
 

  
 
      


# Run

In [ ]:
import importlib
importlib.reload(OmegaConf)

#from ipywidgets.widgets.widget_upload import ButtonStyle


#@markdown **Param Builder**
import base64
import io
parenv=SimpleNamespace(**InitParams())


#####---imgresult----

global wimgresult
global wimgresultretouch

#def drawimgresult(sender):

wimgresult=widgets.Image(
value=parenv.wimgempty,
format='png',
width='auto',
height='auto'
)

wimgresultretouch=widgets.Image(
value=parenv.wimgempty,
format='png',
width=str(parenv.ww.value),
height=str(parenv.wh.value)
)

wimgresultbox=widgets.HBox([wimgresult, wimgresultretouch])

display.display(wimgresultbox)
#888



















In [ ]:
#@markdown **Purge RAM**
Enable_Clean=False #@param {type:"boolean"}
if Enable_Clean: 
  torch.cuda.empty_cache()

In [ ]:
#@title Texto de título predeterminado

testfile=os.path.join(temporal_path,"1.png") 
maskout=os.path.join(temporal_path,"mask.png")
img64=pngtobase64(testfile)
cpaint(maskout,512,512,10,img64)

In [ ]:
#@title Texto de título predeterminado
import ipywidgets as widgets
from ipywidgets import Layout, Button, Box
from IPython.display import display, HTML, Image
from google.colab.output import eval_js
from base64 import b64decode

js_code = '''

function draw() {
  const ctx = document.getElementById('canvas').getContext('2d');
  const img = new Image();
  img.onload = () => {
    ctx.drawImage(img, 0, 0);
    ctx.beginPath();
    ctx.moveTo(30, 96);
    ctx.lineTo(70, 66);
    ctx.lineTo(103, 76);
    ctx.lineTo(170, 15);
    ctx.stroke();
  };
  img.src = 'backdrop.png';
}

draw()
'''
js_code % ()



In [ ]:
#@title <font color='red'>Drawing APP</font> {vertical-output: true, run: "auto"}

import ipywidgets as widgets
from ipywidgets import Layout, Button, Box
from IPython.display import display, HTML, Image
from google.colab.output import eval_js
from base64 import b64decode



Square_Size = 256 #@param ["256", "256"] {type:"raw"}
Brush_Size = 50 #@param {type:"slider", min:0, max:100, step:5}
filename = "/content/temporal/imagen.png"#@param [] {allow-input: true}

js_code = '''
<style>
  .colors-buttons div {
      width: 30px;
      height: 30px;
      margin: 2px;}
  div {
      display: flex;
  }
  canvas{border:1px solid black !important;}
</style>
<canvas id="myCanvas" width="%d" height="%d"></canvas>
<div class="colors-buttons">
  <div class="color" style="background-color: #000000;" id-color="#000000"></div>
  <div class="color" style="background-color: #FFFFFF;" id-color="#FFFFFF"></div>
  <div class="color" style="background-color: #FFFF00;" id-color="#FFFF00"></div>
  <div class="color" style="background-color: #FF00FF;" id-color="#FF00FF"></div>
  <div class="color" style="background-color: #00FFFF;" id-color="#00FFFF"></div>
  <div class="color" style="background-color: #FF0000;" id-color="#FF0000"></div>
  <div class="color" style="background-color: #0000FF;" id-color="#0000FF"></div>
  <div class="color" style="background-color: #00FF00;" id-color="#00FF00"></div>
</div>
<script>
  
  var canvas = document.querySelector('canvas')
  var ctx = canvas.getContext('2d')
  ctx.fillStyle = 'white';
  ctx.fillRect( 0, 0, canvas.width, canvas.height)
  var Brush_Size = %d
  //var img = new Image();
  //img.onload = () => {
  //ctx.drawImage(img, 0, 0);
  //}
  //img.src='https://iili.io/QBYTE7.png';
  var button = document.querySelector('button')
  var mouse = {x: 0, y: 0}
  canvas.addEventListener('mousemove', function(e) {
    mouse.x = e.pageX - this.offsetLeft
    mouse.y = e.pageY - this.offsetTop
  })
  canvas.onmousedown = ()=>{
    ctx.beginPath()
    ctx.moveTo(mouse.x, mouse.y)
    
    canvas.addEventListener('mousemove', onPaint)
  }
  canvas.onmouseup = ()=>{
    canvas.removeEventListener('mousemove', onPaint)
  }
  var onPaint = ()=>{
    ctx.fillRect(mouse.x-( Brush_Size/2), mouse.y-(Brush_Size/2), Brush_Size, Brush_Size)
    ctx.stroke()
  }
  const colors = document.getElementsByClassName('color');
 
  Array.from(colors).forEach(color => {
      color.addEventListener('click', (event) => {
          const colorSelected = event.target.getAttribute('id-color');
          ctx.fillStyle = colorSelected;
      });
  });
    // FINISH BUTTON
  var data = new Promise(resolve=>{
    button.onclick = ()=>{
      resolve(canvas.toDataURL('image/jpg'))
    }
  })
</script>
'''


## Function to Appear Image Canvas
def draw(filename=filename,  w=Square_Size, h=Square_Size, Brush_Size=Brush_Size):

  display(HTML(js_code % (w, h, Brush_Size)))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  if AttributeError:
    pass
  with open(filename, 'wb') as f:
    f.write(binary)
  return len(binary)
  if button2.on_click(on_button_clicked2):
    pass
  #im = im.resize((width // 2, height // 2), resample=Image.ANTIALIAS)

## Action for Reset Button
def on_button_clicked(b):
 
  with output:
   
    #display(HTML(canvas_html % ( w=$Square_Size, h=$Square_Size, Brush_Size=$Brush_Size)))
    data = eval_js("data")
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
      f.write(binary)
  return len(binary)
  

## Show Save Button & Save outputs
button = widgets.Button(description="Save")
button.on_click(on_button_clicked)
output = widgets.Output()
display(button, output)

## Show Canvas for the First Time
draw(filename=filename,  w=Square_Size, h=Square_Size, Brush_Size=Brush_Size)
print("Image Saved at")

In [ ]:
#@title Texto de título predeterminado
from IPython.display import HTML, display
from google.colab import output

# Render the HTML.


html_text = output.eval_js('document.body.innerText')

# %%capture
# print('hola')

In [ ]:
#@title Texto de título predeterminado



#key
#Jr8YHwsSBgwOxm3WrzB1Wv1iiedv2P89RN6GSANqwiw=
#b'&\xbf\x18\x1f\x0b\x12\x06\x0c\x0e\xc6m\xd6\xaf0uZ\xfdb\x89\xe7o\xd8\xff=D\xde\x86H\x03j\xc2,'
#iv
#rwAKlvWf3gK4IIh38oOZng==
#b'\xaf\x00\n\x96\xf5\x9f\xde\x02\xb8 \x88w\xf2\x83\x99\x9e'








from IPython.display import HTML, Image
from google.colab.output import eval_js
from base64 import b64decode
style="border:1px solid #000000"
canvas_html = """
  <canvas width=%d height=%d"></canvas>
  <button>Finish</button>
  <script>
  var canvas = document.querySelector('canvas');
  var ctx = canvas.getContext('2d');
 



  
 
  let newImage = new Image();
  newImage.src = 'file:///content/drive/MyDrive/AI/StableDiffusion/refs/Ref99.png';


  newImage.onload = () => {
    
  ctx.drawImage(newImage, 0, 0, 256, 256);
}

 
  ctx.lineWidth = %d
  ctx.fillStyle = "red"
  var button = document.querySelector('button')
  var mouse = {x: 0, y: 0}
  canvas.addEventListener('mousemove', function(e) {
    mouse.x = e.pageX - this.offsetLeft
    mouse.y = e.pageY - this.offsetTop
  })
  canvas.onmousedown = ()=>{
    ctx.beginPath()
    ctx.moveTo(mouse.x, mouse.y)
    canvas.addEventListener('mousemove', onPaint)
  }
  canvas.onmouseup = ()=>{
    canvas.removeEventListener('mousemove', onPaint)
  }
  var onPaint = ()=>{
    ctx.lineTo(mouse.x, mouse.y)
    ctx.stroke()
  }
  var data = new Promise(resolve=>{
    button.onclick = ()=>{
      resolve(canvas.toDataURL('image/png'))
    }
  })
  </script>
  """

def draw(filename='drawing.png', w=256, h=256, line_width=4):
    display.display(HTML(canvas_html % (w, h, line_width)))
    data = eval_js("data")
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
      f.write(binary)
    return len(binary)

draw()  


####################

#style="border:1px solid #000000
# canvas_html = """
#   <canvas width='256' height='256' style="border:1px solid #000000"></canvas>
#   <button>Finish</button>
#   <script>
#   var canvas = document.querySelector('canvas');
#   var ctx = canvas.getContext('2d');
 
#   var id = ctx.createImageData(1,1); // only do this once per page
#   var d  = id.data;                        // only do this once per page
#   d[0]   = #FF;
#   d[1]   = #FF;
#   d[2]   = 0;
#   d[3]   = 0;
#   ctx.putImageData( id, 1, 1 );  

#   ctx.fillStyle = "#000000";
#   ctx.fillRect(20, 20, 1, 1);
  
#   </script>
#   """


# display.display(HTML(canvas_html))
   


 

# var myImg = new Image();
# img.onload = function() {
#    context.drawImage(myImg, 0, 0);
# };
# img.src = 'https://www.tutorialspoint.com/images/seaborn-4.jpg?v=2';




#   # if it is out of memory, try to use the `--tile` option
#   # We upsample the image with the scale factor X3.5

#   par='-n RealESRGAN_x4plus -i upload -o results --outscale 4 --fp32 --face_enhance'
#   !python inference_realesrgan.py $par
  # Arguments
  # -n, --model_name: Model names
  # -i, --input: input folder or image
  # --outscale: Output scale, can be arbitrary scale factore. 
  #[-h] [-i INPUT] [-n MODEL_NAME] [-o OUTPUT]
  #                               [-dn DENOISE_STRENGTH] [-s OUTSCALE]
  #                               [--model_path MODEL_PATH] [--suffix SUFFIX]
  #                               [-t TILE] [--tile_pad TILE_PAD]
  #                              [--pre_pad PRE_PAD] [--face_enhance] [--fp32]
  #                               [--alpha_upsampler ALPHA_UPSAMPLER] [--ext EXT]
  #                              [-g GPU_ID]

In [ ]:
#@title Texto de título predeterminado
from IPython.display import HTML, Image
from google.colab.output import eval_js
from base64 import b64decode

canvas_html = """
<canvas width=%d height=%d style="border: 1px solid black;"></canvas><br><br>
<button id='clearButton'>Clear</button><br><br>
<button id='circleButton'>Circle</button><br><br>
<button id='drawButton'>Draw</button><br><br>
<button id='creatureButton'>Place Creature</button><br><br>
<label for="radius">Radius of circle:</label><br><br>
<input type="number" value=20 id='radius'><br><br>
<button id='finishButton'>Finish</button>
<script>
var canvas = document.querySelector('canvas')
var ctx = canvas.getContext('2d')
let newImage = new Image();
newImage.onload = () => {
ctx.drawImage(newImage, 0, 0, 256, 256);
}
newImage.src = 'https://iili.io/QBYTE7.png';
   

ctx.lineWidth = %d
var button = document.getElementById('finishButton')
var buttonCircle= document.getElementById('circleButton')
var buttonDraw= document.getElementById('drawButton')
var buttonClear= document.getElementById('clearButton')
var buttonCreature= document.getElementById('creatureButton')
var mouse = {x: 0, y: 0}
var creature_location={x:0,y:0}
var selected='draw'
canvas.addEventListener('mousemove', function(e) {
  mouse.x = e.pageX - this.offsetLeft
  mouse.y = e.pageY - this.offsetTop
})
canvas.addEventListener('mousedown', function(e) {
    if(selected=='draw'){
    ctx.beginPath()
    ctx.moveTo(mouse.x, mouse.y)
    canvas.addEventListener('mousemove', onPaint)
    }
    if(selected=='circle'){
    mouse.x = e.pageX - this.offsetLeft
    mouse.y = e.pageY - this.offsetTop
    ctx.beginPath()
    ctx.arc(mouse.x, mouse.y, document.getElementById('radius').value, 0, 2 * Math.PI)
    ctx.fill()
    }
    if(selected=='place_creature'){
      ctx.clearRect(creature_location.x-20, creature_location.y-20, 40, 40)
      mouse.x = e.pageX - this.offsetLeft
      mouse.y = e.pageY - this.offsetTop
      ctx.fillStyle = "red";
      ctx.beginPath();
      ctx.rect(mouse.x-20, mouse.y-20, 40, 40);
      creature_location.x=mouse.x
      creature_location.y=mouse.y
      ctx.fill();
      ctx.fillStyle = "black";
    }
  
})
canvas.onmouseup = ()=>{
  canvas.removeEventListener('mousemove', onPaint)
}
var onPaint = ()=>{
  ctx.arc(mouse.x, mouse.y, document.getElementById('radius').value, 0, 2 * Math.PI)
  ctx.fill()
  ctx.beginPath()
}
var data = new Promise(resolve=>{
  button.onclick = ()=>{
    resolve(canvas.toDataURL('image/png'))
  }
  buttonCircle.onclick = ()=>{
    selected='circle'
  }
  buttonDraw.onclick = ()=>{
    selected='draw'
  }
  buttonClear.onclick= ()=>{
    ctx.clearRect(0, 0, canvas.width, canvas.height);
  }
  buttonCreature.onclick= ()=>{
    selected='place_creature'
  }
})
</script>
"""

def draw(filename='drawing.png', w=256, h=256, line_width=5):
  a=HTML(canvas_html % (w, h, line_width))
  display(a)
  
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  
    
  return len(binary)
draw()

In [ ]:
#@title <font color='red'>DRAW APP V2</font> {vertical-output: true, run: "auto"}
import ipywidgets as widgets
from IPython.display import display, HTML, Image
from google.colab.output import eval_js
from base64 import b64decode
from datetime import datetime
from termcolor import colored

Square_Size = 256 #@param ["256", "512"] {type:"raw"}
Brush_Size = 60 #@param {type:"slider", min:0, max:100, step:5}
filename = "your_name_here.jpg"

print(colored("Draw the boundary","red"))

canvas_html = """
  <style>
    .colors-buttons div {
        width: 30px;
        height: 30px;
        margin: 1px;
        border: 1px solid black !important;}
    div {
        display: inline-block;
    }
    canvas{border:1px solid black !important;}
  </style>

  <canvas id="myCanvas1" width="%d" height="%d"></canvas>
 

  <div class="colors-buttons">
    <div class="color" style="background-color: #000000;" id-color="#000000"></div>
    <div class="color" style="background-color: #FFFFFF;" id-color="#FFFFFF"></div>
    <div class="color" style="background-color: #FFFF00;" id-color="#FFFF00"></div>
    <div class="color" style="background-color: #FF00FF;" id-color="#FF00FF"></div>
    <div class="color" style="background-color: #00FFFF;" id-color="#00FFFF"></div>
    <div class="color" style="background-color: #FF0000;" id-color="#FF0000"></div>
    <div class="color" style="background-color: #0000FF;" id-color="#0000FF"></div>
    <div class="color" style="background-color: #00FF00;" id-color="#00FF00"></div>
  </div>

  <div>
    <button id="save">Save</button>
    <button id="reset">Reset</button>
    <button id="exit">Exit</button>
    <button id="load">Load</button>
  </div>
  


  <script>




  var canvas = document.getelementbyid('myCanvas')
  var ctx = canvas.getContext('2d')

  ctx.fillStyle = 'white';
  ctx.fillRect( 0, 0, canvas.width, canvas.height)
  var Brush_Size = %d
  let newImage = new Image();
  newImage.onload = () => {
//ctx.drawImage(newImage, 0, 0, 16, 16);
}
newImage.src = 'https://iili.io/QBYTE7.png';


  var button = document.querySelector('button')
  var mouse = {x: 0, y: 0}
  
  var clear_button = document.querySelector('#reset')
  var load=document.querySelector('#load')
  var button = document.querySelector('#save')
  var exit_button = document.querySelector('#exit')
  
  canvas.addEventListener('mousemove', function(e) {
    mouse.x = e.pageX - this.offsetLeft
    mouse.y = e.pageY - this.offsetTop
  })
  canvas.onmousedown = ()=>{
    ctx.beginPath()
    ctx.moveTo(mouse.x, mouse.y)
    
    canvas.addEventListener('mousemove', onPaint)
  }
  canvas.onmouseup = ()=>{
    canvas.removeEventListener('mousemove', onPaint)
  }
  var onPaint = ()=>{

    ctx.fillRect(mouse.x-( Brush_Size/2), mouse.y-(Brush_Size/2), Brush_Size, Brush_Size)
    ctx.stroke()
  }

  const colors = document.getElementsByClassName('color');

  Array.from(colors).forEach(color => {
      color.addEventListener('click', (event) => {
          const colorSelected = event.target.getAttribute('id-color');
          ctx.fillStyle = colorSelected;
      });
  });

   
    clear_button.onclick = ()=>{{
        console.log('Clearing Screen')
        ctx.clearRect(0, 0, canvas.width, canvas.height);
        ctx.fillRect(0, 0, canvas.width, canvas.height);
      }}
      canvas.addEventListener('load', function() {{
      console.log('All assets are loaded')
    }})
    //var data = new Promise(resolve=>{{
    //  button.onclick = ()=>{{
     //   resolve(canvas.toDataURL('image/png'))
    //  }}

 var data = new Promise(resolve=>{{
      load.onclick = ()=>{{
        resolve(newImage.toString())
      }}

      exit_button.onclick = ()=>{{
      resolve()
    }}
      
    }})
        

  </script>
  """


## Function to Appear Image Canvas
def draw(filename=filename,  w=Square_Size, h=Square_Size, Brush_Size=Brush_Size):
  display(HTML(canvas_html % (w, h, Brush_Size)))
  data = eval_js("data")
  print(data)
  # binary = b64decode(data.split(',')[1])
  # if AttributeError:
  #   pass
  # with open(filename, 'wb') as f:
  #   f.write(binary)
  # return len(binary)


  
## Action for Save Button
def on_button_clicked(c):
  #draw(filename=filename,  w=Square_Size, h=Square_Size, Brush_Size=Brush_Size)
  with output:
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Image Saved Again at:", current_time)
  
## Show Save Button & Save outputs
button = widgets.Button(description="Save Image")
button.on_click(on_button_clicked)
output = widgets.Output()
display(button, output)

## Show Canvas for the First Time
draw(filename=filename,  w=Square_Size, h=Square_Size, Brush_Size=Brush_Size)

In [ ]:
#@title Texto de título predeterminado
import base64


In [ ]:




img64=pngtobase64('/content/temporal/1.png')
cpaint('/content/temporal/drawing.png',512,512,10,img64)

In [ ]:
img64=pngtobase64(testfile)
cpaint(maskout,512,512,10,img64)